# Tutorial part 4

In [part 3](part3.ipynb) we have learnt about portfolio states and how to use them in scenario analyses. Here we learn how to export them and how to combine several ones.

## Example data

We start with a similar portfolio state as in the previous part:

In [4]:
import portfolyo as pf
import pandas as pd

index = pd.date_range(
    "2024-09-01", "2024-11-01", freq="15T", inclusive="left", tz="Europe/Berlin"
)
# Creating portfolio line with market prices (here: price-forward curve).
ts_prices = pf.dev.p_marketprices(index, avg=200)
prices = pf.PfLine({"p": ts_prices})


# Creating offtake portfolio line.
ts_offtake = -1 * pf.dev.w_offtake(index, avg=50)
offtake = pf.PfLine({"w": ts_offtake})

# Creating portfolio line with sourced volume.
ts_sourced_power1, ts_sourced_price1 = pf.dev.wp_sourced(
    ts_offtake, "QS", 0.3, p_avg=120
)
sourced_quarters = pf.PfLine({"w": ts_sourced_power1, "p": ts_sourced_price1})
ts_sourced_power2, ts_sourced_price2 = pf.dev.wp_sourced(
    ts_offtake, "MS", 0.2, p_avg=150
)
sourced_months = pf.PfLine({"w": ts_sourced_power2, "p": ts_sourced_price2})
sourced = pf.MultiPfLine(
    {"quarter_products": sourced_quarters, "month_products": sourced_months}
)

# Create the portfolio state.
pfs1 = pf.PfState(offtake, prices, sourced).asfreq('MS')

pfs1.print()

PfState object.
. Timestamps: first: 2024-09-01 00:00:00+02:00     timezone: Europe/Berlin
               last: 2024-10-01 00:00:00+02:00         freq: <MonthBegin> (2 datapoints)
                                                w           q           p             r
                                               MW         MWh     Eur/MWh           Eur
──────── offtake
           2024-09-01 00:00:00 +0200        -49.4     -35 567                          
           2024-10-01 00:00:00 +0200        -54.7     -40 754                          
─●────── pnl_cost
 │         2024-09-01 00:00:00 +0200         49.4      35 567      164.97     5 867 542
 │         2024-10-01 00:00:00 +0200         54.7      40 754      151.18     6 161 235
 ├●───── sourced
 ││        2024-09-01 00:00:00 +0200         21.5      15 445      127.65     1 971 561
 ││        2024-10-01 00:00:00 +0200         38.0      28 324      136.40     3 863 320
 │├───── quarter_products
 ││        2024-09-01 00:00:00 +0200  

## Arithmatic

The final part about portfolio lines is the arithmatic that can be done with them.

Let's create a second portfolio state:

In [7]:
offtake2 = offtake * 2
sourced2 = pf.PfLine(pd.DataFrame({"w": 80, "p": 100}, index))
pfs2 = pf.PfState(offtake2, prices, sourced2).asfreq('MS')
pfs2.print()

PfState object.
. Timestamps: first: 2024-09-01 00:00:00+02:00     timezone: Europe/Berlin
               last: 2024-10-01 00:00:00+02:00         freq: <MonthBegin> (2 datapoints)
                                                w           q           p             r
                                               MW         MWh     Eur/MWh           Eur
──────── offtake
           2024-09-01 00:00:00 +0200        -98.8     -71 134                          
           2024-10-01 00:00:00 +0200       -109.4     -81 509                          
─●────── pnl_cost
 │         2024-09-01 00:00:00 +0200         98.8      71 134      118.75     8 447 101
 │         2024-10-01 00:00:00 +0200        109.4      81 509      131.96    10 756 297
 ├────── sourced
 │         2024-09-01 00:00:00 +0200         80.0      57 600      100.00     5 760 000
 │         2024-10-01 00:00:00 +0200         80.0      59 600      100.00     5 960 000
 └────── unsourced
           2024-09-01 00:00:00 +0200         

Note that ``pfs1`` and ``pfs2`` have distinct unsourced prices at this month level, even though they were created using the same market prices on the quarter-hour level.

### Addition and subtraction

We can add these two portfolio states:

In [8]:
pfs1 + pfs2

c:\users\ruud.wijtvliet\ruud\python\dev\portfolyo\portfolyo\core\pfline\enable_arithmatic.py:82: PfLineFlattenedWarning: When adding a SinglePfLine and MultiPfLine, the MultiPfLine is flattened.
  warnings.warn(


ValueError: Need at least 3 dates to infer frequency

Note that the individual components are added together. The volumes (offtake, sourced, unsourced) are summed; the prices (sourced and unsourced) are the energy-weighted averaged. (Or, put differently, the *revenues* are also summed, and the prices are calculated from the volume-total and renevue-total.)

Note also that the sourced volume of ``pfs1`` has been flattened, i.e., the values of its children are lost. This is because ``pfs2`` does not have any children. This behaviour is described [here](../core/pfline.rst#Arithmatic).

Likewise we can subtract them with ``pfs1 - pfs2``:


In [9]:
pfs1 - pfs2

c:\users\ruud.wijtvliet\ruud\python\dev\portfolyo\portfolyo\core\pfline\enable_arithmatic.py:82: PfLineFlattenedWarning: When adding a SinglePfLine and MultiPfLine, the MultiPfLine is flattened.
  warnings.warn(


ValueError: Need at least 3 dates to infer frequency